In [15]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
# natural language toolkit for textual data
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
df=pd.read_csv(r"C:\Users\karni\OneDrive\Desktop\chrome downloads\Sem6\Minor_Project\Cleaned_data.csv",encoding='ISO-8859-1')

In [17]:
df.shape

(1600000, 8)

In [18]:
df.head()

,Unnamed: 0,target,id,date,flag,user,text,stemmed_content
0,0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [19]:
#counting the number missing values in the dataset
df.isnull().sum()

Unnamed: 0           0
target               0
id                   0
date                 0
flag                 0
user                 0
text                 0
stemmed_content    495
dtype: int64

In [20]:
#checking the distribution of target column
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [21]:
# print(df['stemmed_content'])
print(df['stemmed_content'].isnull().sum())
# Drop rows with NaN values
df.dropna(subset=['stemmed_content'], inplace=True)
print(df['stemmed_content'].isnull().sum())
df.shape
df=df[::2000]

495
0


In [22]:
x=df['stemmed_content'].values
y=df['target'].values

In [23]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'morn twitterati larg coffe bath top today list first appt load desk work'
 'what world ppl decid shoot other watch news depress'
 'new xserv new higher price surpris'
 'higher cop donut ratio today left us'
 'nawong go get access idzr org die tri'
 'still sick still without pair brown plaid green nike blazer men'
 'hate go school'
 'today hung friend noth much guitar amp waaaaay bust back school monday god take life'
 'brother understand beauti headbang sad' 'tri fix phone fail'
 'hamlet romeo n juliet radio activ live wembley mcfli tour dvd money'
 'back studi tell guy bore studi exam rather lab report'
 'watch wow wow wubzi logan say goodby shandi im go miss'
 'bookreport stalk subject dutch make senc know'
 'want go back sixth form today' 'set alarm realiz go go less hour'
 'raspberri white chocol cupcak hit poor poor henrywoodhous peep'
 'think might go bed certain someon start make sens matter much pa

In [24]:
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [25]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2,stratify=y)

In [26]:
print(x.shape,x_train.shape,x_test.shape)

(800,) (640,) (160,)


In [27]:
#converting the actual data to numerical data
vectorizer=TfidfVectorizer()
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

In [28]:
from pyswarm import pso
# Define the objective function for PSO
def objective_function(x):
    C = 10 ** x[0]  # Assuming x[0] represents the log10(C)
    gamma = 10 ** x[1]  # Assuming x[1] represents the log10(gamma)
    model=LogisticRegression(max_iter=1000)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return -accuracy_score(y_test, y_pred)  # Negative since PSO minimizes

# Set up the PSO optimizer
num_dimensions = 2  # Number of dimensions in the search space (C and gamma)
# lb = [-4, -4]  # Lower bounds for C and gamma
# ub = [4, 4]  # Upper bounds for C and gamma
#above line giving 60% accuracy
lb = [-1, -10]  
ub = [1, 10] 


x_opt, f_opt = pso(objective_function, lb, ub, swarmsize=20, maxiter=100)

# Convert the optimal solution to C and gamma values
C = 10 ** x_opt[0]
gamma = 10 ** x_opt[1]
# Train the final model using the best hyperparameters
final_model = LogisticRegression()
final_model.fit(x_train, y_train)
y_pred = final_model.predict(x_test)
final_accuracy = accuracy_score(y_test, y_pred)
print("Best hyperparameters (C, gamma):", (C, gamma))
print("Best accuracy found by PSO:", -f_opt)
print("Accuracy of the final model:", final_accuracy)

Stopping search: maximum iterations reached --> 100
Best hyperparameters (C, gamma): (0.21712653734211754, 1900386.1137381205)
Best accuracy found by PSO: 0.6375
Accuracy of the final model: 0.6375


In [29]:
# model=LogisticRegression(max_iter=1000)
# model.fit(x_train,y_train)

In [30]:
# #accuracy score on the training data
# x_train_pred=model.predict(x_train)
# train_data_acc=accuracy_score(y_train,x_train_pred)

In [31]:
# print("Training Data Accuracy",train_data_acc)

In [32]:
# #accuracy score on the testing data
# x_test_pred=model.predict(x_test)
# test_data_acc=accuracy_score(y_test,x_test_pred)

In [33]:
# print("Testing Data Accuracy",test_data_acc)

In [34]:
print("Classification Report:")
print(classification_report(y_test, x_test_pred))

Classification Report:


NameError: name 'x_test_pred' is not defined

In [ ]:
#Saving the trained Model
import pickle

In [ ]:
filename="sentimental_analysis_trained_mode.sav"
pickle.dump(model,open(filename,'wb'))

In [ ]:
#Using the saved model for future predictions
loaded_model=pickle.load(open(r"C:\Users\karni\OneDrive\Desktop\chrome downloads\Sem6\Minor_Project\sentimental_analysis_trained_mode.sav",'rb'))

In [ ]:
x_new=x_test[200]
print(y_test[200])
prediction=model.predict(x_new)
print(prediction)
if(prediction[0]==0):
  print("Negative Tweet")
else:
  print("Positive Tweet")

0
[0]
Negative Tweet


In [ ]:
x_new=x_test[3]
print(y_test[3])
prediction=model.predict(x_new)
print(prediction)
if(prediction[0]==0):
  print("Negative Tweet")
else:
  print("Positive Tweet")

0
[0]
Negative Tweet


In [ ]:
X = df['stemmed_content']
y = df['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer=TfidfVectorizer()
X_train_vec=vectorizer.fit_transform(X_train)
X_test_vec=vectorizer.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the classifier
knn.fit(X_train_vec, y_train)

# Make predictions
y_pred = knn.predict(X_test_vec)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5907522256677004
